#Pré-processamento dos dados da base de dados CPD_Parks.csv

In [ ]:
# Importação das Bibliotecas Necessárias
import pandas as pd
import shapely.wkt
import geopandas as gpd
from geopy.distance import geodesic
import numpy as np


In [ ]:
#carregar od Dados
# Foi tranfomado a coluna'the_geom' de WKT para objetos geométricos utilizando a biblioteca shapely.

df = pd.read_csv("CPD_Parks.csv")
df['geometry'] = df['the_geom'].apply(lambda x: shapely.wkt.loads(x))

In [ ]:
# Conversão para GeoDataFrame

gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf = gdf.set_crs('epsg:4326')

In [ ]:
# Extração de Latitude e Longitude

df['Latitude'] = gdf['geometry'].apply(lambda geom: geom.centroid.y if geom.geometryType() in ['Polygon', 'MultiPolygon'] else geom.y)
df['Longitude'] = gdf['geometry'].apply(lambda geom: geom.centroid.x if geom.geometryType() in ['Polygon', 'MultiPolygon'] else geom.x)

In [ ]:
# Função para Calcular Distância entre dois pontos geográficos em quilômetros.
def calcular_distancia(coord1, coord2):
    return geodesic(coord1, coord2).km

In [ ]:
# Análise de Distância entre Parques e Pontos de Interesse
# Foi calculada a distância mínima para um parque, onde foi verificado a existência de parques dentro de um intervalo específico



entre2e10 = []
dist_minimas = []
dist_maior_2km = []
saber=''
for i in range(len(db)):
    lat1, long1 = db.loc[i, 'LAT'], db.loc[i, 'LONG']
    coord1 = (lat1, long1)
    dist_min = np.inf
    dist_maior = 0
    for j in range(len(df)):
        lat2, long2 = df.loc[j, 'Latitude'], df.loc[j, 'Longitude']
        coord2 = (lat2, long2)
        dist = calcular_distancia(coord1, coord2)
        if dist < dist_min:
            dist_min = dist
        if dist > 2 and dist > dist_maior and dist < 10:
            dist_maior = dist
        if dist >= 2 and dist <= 10:
            saber='1'
        else:
            saber='0'
    entre2e10.append(saber)
    dist_minimas.append(dist_min)
    dist_maior_2km.append(dist_maior)

db['Menor Distancia'] = dist_minimas
db['Existencia de Pracas'] = entre2e10
db['Maior Distancia > 2km'] = dist_maior_2km